In [1]:
import transformer_lens
from transformer_lens import utils
import torch
import tqdm
from utils import *
# Load a model (eg GPT-2 Small)
model = transformer_lens.HookedTransformer.from_pretrained("gpt2-small")

# Run the model and get logits and activations
logits, activations = model.run_with_cache("The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid")

logits.sum(-1)

/Users/pavan/miniconda3/envs/interp/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/pavan/miniconda3/envs/interp/lib/python3.11/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


Using pad_token, but it is not set yet.


Loaded pretrained model gpt2-small into HookedTransformer


tensor([[-0.0308, -0.0698, -0.0879, -0.0766, -0.1271, -0.1322, -0.0886, -0.1147,
         -0.1190, -0.1054, -0.1420, -0.1411, -0.1529, -0.1618, -0.1431, -0.1082,
         -0.1749, -0.0615, -0.0832, -0.0933, -0.1631, -0.1422, -0.1095]],
       device='mps:0', grad_fn=<SumBackward1>)

In [2]:
activations.keys()

dict_keys(['hook_embed', 'hook_pos_embed', 'blocks.0.hook_resid_pre', 'blocks.0.ln1.hook_scale', 'blocks.0.ln1.hook_normalized', 'blocks.0.attn.hook_q', 'blocks.0.attn.hook_k', 'blocks.0.attn.hook_v', 'blocks.0.attn.hook_attn_scores', 'blocks.0.attn.hook_pattern', 'blocks.0.attn.hook_z', 'blocks.0.hook_attn_out', 'blocks.0.hook_resid_mid', 'blocks.0.ln2.hook_scale', 'blocks.0.ln2.hook_normalized', 'blocks.0.mlp.hook_pre', 'blocks.0.mlp.hook_post', 'blocks.0.hook_mlp_out', 'blocks.0.hook_resid_post', 'blocks.1.hook_resid_pre', 'blocks.1.ln1.hook_scale', 'blocks.1.ln1.hook_normalized', 'blocks.1.attn.hook_q', 'blocks.1.attn.hook_k', 'blocks.1.attn.hook_v', 'blocks.1.attn.hook_attn_scores', 'blocks.1.attn.hook_pattern', 'blocks.1.attn.hook_z', 'blocks.1.hook_attn_out', 'blocks.1.hook_resid_mid', 'blocks.1.ln2.hook_scale', 'blocks.1.ln2.hook_normalized', 'blocks.1.mlp.hook_pre', 'blocks.1.mlp.hook_post', 'blocks.1.hook_mlp_out', 'blocks.1.hook_resid_post', 'blocks.2.hook_resid_pre', 'block

In [5]:
line(activations['blocks.4.mlp.hook_post'][0,1,:])

In [14]:
activations['blocks.4.mlp.hook_post'][activations['blocks.4.mlp.hook_post']>0].shape

torch.Size([11162])

In [50]:

line(activations['blocks.4.mlp.hook_post'][0,:,0])
line(activations['blocks.4.mlp.hook_post'][0,:,0])

In [26]:
activations['blocks.4.mlp.hook_post'].shape

torch.Size([1, 23, 3072])

In [55]:
(activations['blocks.4.mlp.hook_post']>0)[0][0].shape

torch.Size([3072])

In [56]:
model.cfg

HookedTransformerConfig:
{'act_fn': 'gelu_new',
 'attention_dir': 'causal',
 'attn_only': False,
 'attn_types': None,
 'checkpoint_index': None,
 'checkpoint_label_type': None,
 'checkpoint_value': None,
 'd_head': 64,
 'd_mlp': 3072,
 'd_model': 768,
 'd_vocab': 50257,
 'd_vocab_out': 50257,
 'default_prepend_bos': True,
 'device': device(type='mps'),
 'dtype': torch.float32,
 'eps': 1e-05,
 'final_rms': False,
 'from_checkpoint': False,
 'gated_mlp': False,
 'init_mode': 'gpt2',
 'init_weights': False,
 'initializer_range': 0.02886751345948129,
 'model_name': 'gpt2',
 'n_ctx': 1024,
 'n_devices': 1,
 'n_heads': 12,
 'n_layers': 12,
 'n_params': 84934656,
 'normalization_type': 'LNPre',
 'original_architecture': 'GPT2LMHeadModel',
 'parallel_attn_mlp': False,
 'positional_embedding_type': 'standard',
 'rotary_dim': None,
 'scale_attn_by_inverse_layer_idx': False,
 'seed': None,
 'tokenizer_name': 'gpt2',
 'tokenizer_prepends_bos': False,
 'use_attn_in': False,
 'use_attn_result': Fals

In [2]:
from torch import nn
import os, sys
import torch.nn.functional as F


from transformer_lens import HookedTransformer, utils
import torch
from datasets import load_dataset
from typing import Dict
from tqdm.notebook import tqdm
import plotly.express as px
import json

device = torch.device("cpu")

torch.set_grad_enabled(False)

def imshow(x, **kwargs):
    x_numpy = utils.to_numpy(x)
    px.imshow(x_numpy, **kwargs).show()
DTYPES = {"fp32": torch.float32, "fp16": torch.float16, "bf16": torch.bfloat16}
class AutoEncoder(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.cfg = cfg
        d_hidden = cfg["dict_size"]
        l1_coeff = cfg["l1_coeff"]
        dtype = DTYPES[cfg["enc_dtype"]]
        torch.manual_seed(cfg["seed"])
        self.W_enc = nn.Parameter(torch.nn.init.kaiming_uniform_(torch.empty(cfg["act_size"], d_hidden, dtype=dtype)))
        self.W_dec = nn.Parameter(torch.nn.init.kaiming_uniform_(torch.empty(d_hidden, cfg["act_size"], dtype=dtype)))
        self.b_enc = nn.Parameter(torch.zeros(d_hidden, dtype=dtype))
        self.b_dec = nn.Parameter(torch.zeros(cfg["act_size"], dtype=dtype))

        self.W_dec.data[:] = self.W_dec / self.W_dec.norm(dim=-1, keepdim=True)

        self.d_hidden = d_hidden
        self.l1_coeff = l1_coeff
        self.dtype = dtype
        self.device = device


        self.version = 0
        self.to(self.device)

    def forward(self, x, per_token=False):
        x_cent = x - self.b_dec
        acts = F.relu(x_cent @ self.W_enc + self.b_enc) # [batch_size, d_hidden]
        x_reconstruct = acts @ self.W_dec + self.b_dec # [batch_size, act_size]
        if per_token:
            l2_loss = (x_reconstruct.float() - x.float()).pow(2).sum(-1) # [batch_size]
            l1_loss = self.l1_coeff * (acts.float().abs().sum(dim=-1)) # [batch_size]
            loss = l2_loss + l1_loss # [batch_size]
        else:
            l2_loss = (x_reconstruct.float() - x.float()).pow(2).sum(-1).mean(0) # []
            l1_loss = self.l1_coeff * (acts.float().abs().sum(dim=-1).mean(dim=0)) # []
            loss = l2_loss + l1_loss # []
        return loss, x_reconstruct, acts, l2_loss, l1_loss


    @classmethod
    def load_from_hf(cls, version, hf_repo="ckkissane/tinystories-1M-SAES"):
        """
        Loads the saved autoencoder from HuggingFace.
        """

        cfg = utils.download_file_from_hf(hf_repo, f"{version}_cfg.json")
        self = cls(cfg=cfg)
        self.load_state_dict(utils.download_file_from_hf(hf_repo, f"{version}.pt", force_is_torch=True))
        return self

/Users/pavan/miniconda3/envs/interp/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/pavan/miniconda3/envs/interp/lib/python3.11/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


In [3]:
# Layer 9
auto_encoder_run = "gpt2-small_L1_Hcat_z_lr1.20e-03_l18.00e-01_ds24576_bs4096_dc1.00e-06_rsanthropic_rie25000_nr4_v5"

encoder = AutoEncoder.load_from_hf(auto_encoder_run, hf_repo="ckkissane/attn-saes-gpt2-small-all-layers")
model = HookedTransformer.from_pretrained(encoder.cfg["model_name"]).to(DTYPES[encoder.cfg["enc_dtype"]]).to(device)

Using pad_token, but it is not set yet.


Loaded pretrained model gpt2-small into HookedTransformer
Changing model dtype to torch.float32
Moving model to device:  cpu


In [4]:
import einops
def get_batch_tokens(dataset_iter, batch_size, model, cfg):
    tokens = []
    total_tokens = 0
    while total_tokens < batch_size*cfg["seq_len"]:
        try:
            # Retrieve next item from iterator
            row = next(dataset_iter)["text"]
        except StopIteration:
            # Break the loop if dataset ends
            break

        # Tokenize the text with a check for max_length
        cur_toks = model.to_tokens(row)
        tokens.append(cur_toks)

        total_tokens += cur_toks.numel()

    # Check if any tokens were collected
    if not tokens:
        return None

    # Depending on your model's tokenization, you might need to pad the tokens here

    # Flatten the list of tokens
    flat_tokens = torch.cat(tokens, dim=-1).flatten()
    flat_tokens = flat_tokens[:batch_size * cfg["seq_len"]]
    reshaped_tokens = einops.rearrange(
        flat_tokens,
        "(batch seq_len) -> batch seq_len",
        batch=batch_size,
    )
    reshaped_tokens[:, 0] = model.tokenizer.bos_token_id
    return reshaped_tokens

def shuffle_data(all_tokens):
    print("Shuffled data")
    return all_tokens[torch.randperm(all_tokens.shape[0])]

dataset = load_dataset('Skylion007/openwebtext', split='train', streaming=True)
dataset_iter = iter(dataset)

all_tokens_batches = int(1e6) // encoder.cfg["seq_len"]



print("Data was not cached: Loading data first time")
all_tokens = get_batch_tokens(dataset_iter, all_tokens_batches, model, encoder.cfg)


/Users/pavan/miniconda3/envs/interp/lib/python3.11/site-packages/datasets/load.py:1429: FutureWarning: The repository for Skylion007/openwebtext contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/Skylion007/openwebtext
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
Data was not cached: Loading data first time
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [5]:
model.to_string(all_tokens[5])

'<|endoftext|>He and the others stayed with the injured all night, after the medical team had left and after the generators gave out and the tents turned pitch black.\n\nGupta monitored patients\' vital signs, administered painkillers and continued intravenous drips. He stabilized three new patients in critical condition.\n\nAt 3:45 a.m., he posted a message on Twitter: "pulling all nighter at haiti field hosp. lots of work, but all patients stable. turned my crew into a crack med team tonight."\n\nAre you in Haiti and safe? Share your photos\n\nHe said the Belgian doctors did'

In [6]:
all_tokens.shape

torch.Size([7812, 128])

In [1]:
batch_size = 16
layer_idx = 1
acts_key = 'blocks.1.attn.hook_z'
batches = 2
context_length = 128

In [2]:
from tqdm import tqdm
attn_acts = torch.tensor([], device= device)
expanded_acts = torch.tensor([], device= device)

with torch.no_grad():
    for i in tqdm(range(batches)):
        logits, cache = model.run_with_cache(all_tokens[i*batch_size:(i+1)*batch_size],stop_at_layer = layer_idx +1)
        attn = einops.rearrange(
                cache[acts_key][:,:context_length-1, :,:], "b seq n_heads d_head -> (b seq) (n_heads d_head)",
            )
        attn_acts = torch.cat((attn_acts, attn))
        

print(attn_acts.shape)        

NameError: name 'torch' is not defined

In [ ]:
loss = torch.tensor([], device= device)

with torch.no_grad():
    for i in  tqdm(range(batches)):
        curr_loss = model(all_tokens[i*batch_size:(i+1)*batch_size], return_type = 'loss', loss_per_token = True).view(-1)
        loss = torch.cat((loss, curr_loss))
        
print(loss.shape)

torch.Size([130048])


In [ ]:
torch.save(loss, "loss.pt")
torch.save(expanded_acts, "expanded_acts.pt")
torch.save(attn_acts, "attn_acts.pt")


In [81]:
b,c, d = expanded_acts.shape

In [87]:
expanded_acts = einops.rearrange(expanded_acts[:,0:c-1,:], "b c d -> d b c")

In [85]:
from utils import *
line(expanded_acts.sum(dim = 0).sum(dim=0))

In [13]:
loss.shape

torch.Size([130048])

In [14]:
expanded_acts.shape

torch.Size([130048, 24576])

In [15]:
index = 1952

for index in range(50):
    a = expanded_acts[:, index]
    if a.sum()>0:
        print(index, loss[a>0].mean(), (a>0).sum().item())

1 tensor(2.8597) 3
2 tensor(1.9784) 6
3 tensor(4.2283) 3
6 tensor(3.3318) 29
7 tensor(1.1063) 2
9 tensor(3.1620) 44
10 tensor(4.1066) 4
11 tensor(3.3954) 1170
13 tensor(3.6989) 173
14 tensor(3.8538) 17
15 tensor(1.0295) 10
16 tensor(2.6808) 6
17 tensor(2.4720) 211
21 tensor(2.9563) 41
22 tensor(2.9888) 304
30 tensor(3.7238) 864
32 tensor(2.5865) 1
33 tensor(2.4995) 8
43 tensor(5.4285) 2
44 tensor(1.7040) 18
46 tensor(3.1248) 8
47 tensor(4.7722) 15
48 tensor(3.4811) 5
49 tensor(3.0790) 65


In [25]:
expanded_acts[:, 13].argmax() , loss

(tensor(41203), tensor([8.6271, 5.4034, 2.4274,  ..., 3.3521, 0.3561, 0.7994]))

In [26]:
loss[41203]

tensor(3.9584)